## Imports

In [1]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
from mog_model import *
from utils.distributions import *
from lattice.ising_lattice import IsingLattice
from lattice.gauge_lattice import GaugeLattice
from lattice.fields import SU2, SU3
from utils.dynamics import *
from utils.gauge_dynamics import GaugeDynamics
from utils.func_utils import accept, jacobian, autocovariance, get_log_likelihood, binarize, normal_kl, acl_spectrum, ESS
from utils.network import network, Linear, Sequential, Zip, Parallel, ScaleTanh
from utils.sampler import propose
from utils.notebook_utils import get_hmc_samples
from utils.data_utils import calc_avg_vals_errors, block_resampling, jackknife_err
from matplotlib import gridspec
from utils.plot_helper import errorbar_plot
plt.style.use('/Users/saforem2/.config/matplotlib/stylelib/dark_jupyter.mplstyle')

plt.rc('text', usetex=False)
plt.rcParams['errorbar.capsize'] = 0
%matplotlib notebook
%autoreload 2

In [2]:
sess = tf.InteractiveSession()

In [3]:
def restart_tf(sess):
    tf.reset_default_graph()
    sess.close()
    return tf.InteractiveSession()

## Create GaugeLattice

In [9]:
sess = restart_tf(sess)

In [10]:
time_size = 2
space_size = 4
dim = 4
beta = 1.,
link_type = 'SU3' 
lattice = GaugeLattice(time_size, space_size, dim, beta, link_type)
# create `num_samples` random samples of GaugeLattice.links
links_samples = lattice.get_links_samples(5, link_type='SU3')

In [11]:
dynamics = GaugeDynamics(lattice, trajectory_length=2, eps=0.1,
                         hmc=False, net_factory=network, eps_trainable=True,
                         use_temperature=True)

In [12]:
batch_size = 5
link = tf.placeholder(dtype=tf.complex64, shape=lattice.link_shape, name='link')
x_shape = tuple([batch_size] + list(lattice.links.shape))
x = tf.placeholder(tf.complex64, shape=x_shape, name='x')
z = tf.random_normal(tf.shape(x), name='z')

In [13]:
energy = dynamics.energy(x, batch_size)

In [14]:
energy_arr = sess.run(energy, feed_dict={x: links_samples, dynamics.temperature: 1.})

In [15]:
energy_arr

array([1267.7571, 1267.7401, 1268.4408, 1267.8602, 1268.0498],
      dtype=float32)

In [31]:
H = dynamics.hamiltonian(x, z)
print(sess.run(H), feed_dict={x: samples, dynamics.temperature: 1.})

InvalidArgumentError: You must feed a value for placeholder tensor 'temperature' with dtype float
	 [[Node: temperature = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'temperature', defined at:
  File "/Users/saforem2/miniconda3/envs/tensorflow_cpu/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/saforem2/miniconda3/envs/tensorflow_cpu/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/saforem2/miniconda3/envs/tensorflow_cpu/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/saforem2/miniconda3/envs/tensorflow_cpu/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/saforem2/miniconda3/envs/tensorflow_cpu/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start(

In [17]:
step1 = dynamics._forward_step(x, z, 1)

TypeError: energy() got an unexpected keyword argument 'aux'

In [75]:
%debug

> /Users/saforem2/miniconda3/envs/tensorflow_cpu/lib/python3.6/site-packages/tensorflow/python/framework/op_def_library.py(788)_apply_op_helper()
    786                          input_types=input_types, attrs=attr_protos,
    787                          op_def=op_def)
--> 788       return output_structure, op_def.is_stateful, op
    789 
    790 # pylint: enable=invalid-name

ipdb> ll
    339   def _apply_op_helper(self, op_type_name, name=None, **keywords):
    340     """Implementation of apply_op that returns output_structure, op."""
    341     op_info = self._ops.get(op_type_name, None)
    342     if op_info is None:
    343       raise RuntimeError("Unrecognized Op name " + op_type_name)
    344     op_def = op_info.op_def
    345 
    346     # Determine the graph context.
    347     try:
    348       # Need to flatten all the arguments into a list.
    349       # pylint: disable=protected-access
    350       g = ops._get_graph_from_inputs(_Flatten(keywords.values()))
   

ipdb> up
> /Users/saforem2/miniconda3/envs/tensorflow_cpu/lib/python3.6/site-packages/tensorflow/python/ops/gen_math_ops.py(4456)mat_mul()
   4454     _, _, _op = _op_def_lib._apply_op_helper(
   4455         "MatMul", a=a, b=b, transpose_a=transpose_a, transpose_b=transpose_b,
-> 4456         name=name)
   4457     _result = _op.outputs[:]
   4458     _inputs_flat = _op.inputs

ipdb> up
> /Users/saforem2/miniconda3/envs/tensorflow_cpu/lib/python3.6/site-packages/tensorflow/python/ops/math_ops.py(2018)matmul()
   2016     else:
   2017       return gen_math_ops.mat_mul(
-> 2018           a, b, transpose_a=transpose_a, transpose_b=transpose_b, name=name)
   2019 
   2020 

ipdb> up
> /Users/saforem2/ANL/l2hmc/l2hmc/utils/network.py(124)__call__()
    122         #self.W = tf.reshape(self.W, shape=tuple([x.shape] + [-1]))
    123         if x.dtype == tf.complex64:
--> 124             self.activations = tf.add(complex_matmul(x, self.W),
    125                                       tf.ca

In [ ]:
Lx, _, px, output = propose(x, dynamics, do_mh_step=True)

In [ ]:
Lz, _, pz, _ = propose(z, dynamics, do_mh_step=True)

loss = tf.Variable(0., trainable=False, name='loss')
v1 = ((tf.reduce_sum(tf.square(x - Lx), axis=1) * px) + 1e-4)
v2 = ((tf.reduce_sum(tf.square(z - Lz), axis=1) * pz) + 1e-4)
scale = 0.1
loss = loss + scale * (tf.reduce_mean(1.0 / v1) + tf.reduce_mean(1.0 / v2))
loss = loss + ((-tf.reduce_mean(v1, name='v1')
                - tf.reduce_mean(v2, name='v2')) / scale)

In [ ]:
with tf.Session() as sess:
    print(sess.run(lattice.total_action(lattice.links)))

In [ ]:
energy = dynamics.energy(x)
with tf.Session() as sess:
    print(sess.run(energy, feed_dict={x: links_samples, dynamics.temperature:1.}))

## LatticeDynamics testing

### Init tf.Session()

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.close()

In [ ]:
sess = tf.InteractiveSession()

In [ ]:
init = tf.global_variables_initializer()

In [ ]:
sess.run(tf.global_variables_initializer())

### Create LatticeDynamics object

In [ ]:
_x_shape = tuple([None] + list(lattice.links.shape) + list(lattice.link_shape))
_x = tf.placeholder(dtype=tf.complex64, shape=_x_shape)

_links_samples = []
for _ in range(10):
    _links_sample = np.zeros(_x_shape[1:], dtype=np.complex64)
    for link in lattice.iter_links():
        _links_sample[link] = lattice.link_type.get_random_element()
    _links_samples.append(_links_sample)

In [ ]:
lattice.num_links

In [ ]:
_mask = sess.run(dynamics.mask)

In [ ]:
_mask.reshape(tuple([-1] + list(lattice.links.shape)))

In [ ]:
m, _m = dynamics._get_mask(0)

In [ ]:
np.array(links_samples).astype(np.complex64)

In [ ]:
links_samples = np.array(np.array(links_samples), dtype=np.complex64)

In [ ]:
kinetic = dynamics.kinetic(z)
print(sess.run(kinetic), feed_dict={x: np.array(links_samples), dynamics.temperature: 1.})

In [ ]:
hamiltonian = dynamics.hamiltonian(x, z)
sess.run(hamiltonian, feed_dict{x:})

In [ ]:
temp = np.array([1.0], dtype=np.float32)[0]
energy = dynamics.energy(x)
print(sess.run(energy, feed_dict={x: links_samples, dynamics.temperature: temp}))

In [ ]:
print(sess.run(energy, feed_dict={x: links_samples, dynamics.temperature: temp}))

In [ ]:
print(sess.run(dynamics.lattice.total_action()))
#dynamics.lattice.total_action()

In [ ]:
forward_step = dynamics._forward_step(x, z, 1, aux=None)

In [ ]:
%debug

In [ ]:
step1 = sess.run(forward_step, feed_dict={x: samples, dynamics.temperature: 2.})

In [ ]:
x1, v1, log_jac_contrib1 = step1

In [ ]:
energy = dynamics.energy(x)
print(sess.run(energy, feed_dict={x: samples, dynamics.temperature:2.}))
print(sess.run(energy, feed_dict={x: x1, dynamics.temperature:2.}))

In [ ]:
hamiltonian = dynamics.hamiltonian(x, z)
print(sess.run(hamiltonian, feed_dict={x: samples, dynamics.temperature:2.}))
print('\n')
print(sess.run(hamiltonian, feed_dict={x: x1, dynamics.temperature:2.}))

In [ ]:
grad_energy = dynamics.grad_energy(x)
print(sess.run(grad_energy, feed_dict={x: samples, dynamics.temperature:2.}))
print('\n')
print(sess.run(grad_energy, feed_dict={x: x1, dynamics.temperature:2.}))

In [ ]:
X, V, p_accept = dynamics.forward(x)

In [ ]:
_X = sess.run(X, feed_dict={x: samples, dynamics.temperature:2.})
print(_X.shape)

In [ ]:
_V = sess.run(V, feed_dict={x: samples, dynamics.temperature:2.})
print(_V.shape)

In [ ]:
_p_accept = sess.run(p_accept, feed_dict={x: samples, dynamics.temperature:2.})
print(_p_accept.shape)
print(_p_accept)